 # Influencer Recommendation Engine

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import json

import tweepy as tw
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import networkx as nx

import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Get access to Twitter API

In [2]:
consumer_key = '8RC8HWJwtgTML1oUrZnIBmC42'
consumer_secret = '2hvnUhs3RwvRLEhdx2xvHGdwQkCGUuZXN9VFEnaomcH4qJAmkG'
access_token = '1266466384449601536-ZwDr9eixclZckMQzZ3ZOScaFMNWe49'
access_token_secret = 'pXNlLrHFBWtehr1j4eBwVgAOhzDmCKiL08rjST27bNcrt'

# Authenticate to Twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)
# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


 # Get tweets based on specific search

In [21]:
# Define the search term and the date_since date as variables
search_words = "$CELH -filter:retweets"
date_since = "2019-06-01"

In [22]:
# Collect tweets
tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since).items(10)

In [23]:
all_tweets = [tweet.text for tweet in tweets]

In [24]:
all_tweets[1]

'$WKHS CFO interview 7/7/2020!! Major USPS mention!! .40c vs $1 per mile average cost!!!! THEY ARE GETTING THIS CONT… https://t.co/jiSnLhGgWy'

# Get all tweets from a user

In [18]:
tweet_texts = []
timestamps = []
users = []
  
try:  
    for tweet in tw.Cursor(api.user_timeline, screen_name="nygovcuomo", exclude_replies=True, count = 10).items(10):  
                    tweet_text = tweet.text  
                    time = tweet.created_at  
                    tweeter = tweet.user.screen_name  
                    tweet_texts.append(tweet_text.strip())
                    timestamps.append(time)
                    users.append(tweeter) 
except tw.TweepError:  
    time.sleep(60)

In [26]:
df_tweets = pd.DataFrame(columns = ['tweets_texts', 'timestamps', 'users'])

In [27]:
df_tweets.tweets_texts = tweet_texts
df_tweets.timestamps = timestamps
df_tweets.users = users

In [28]:
df_tweets.head()

,tweets_texts,timestamps,users
0,"Today's update on the numbers:\n\n56,736 tests...",2020-07-07 14:00:49,NYGovCuomo
1,We are saddened by the death of Spc. Vincent S...,2020-07-06 21:34:44,NYGovCuomo
2,"Not knowing, not testing is not an answer.\n\n...",2020-07-06 19:19:03,NYGovCuomo
3,No mammograms? We cured breast cancer.\n\nNo H...,2020-07-06 16:13:35,NYGovCuomo
4,Violations of social distancing and mask-weari...,2020-07-06 15:58:16,NYGovCuomo


 # Get the followers of a user

In [13]:
try:   
    followers = api.followers_ids(screen_name="mikekobe0")  
except tweepy.TweepError:  
    time.sleep(20)

In [14]:
df_followers = pd.DataFrame(columns = ['user_number'], data = followers)

In [15]:
def get_user_name(user_number):
    user = api.get_user(user_number)
    return user.screen_name

In [16]:
df_followers['user_name'] = df_followers['user_number'].apply(get_user_name)

In [17]:
df_followers.head()

,user_number,user_name
0,2319965522,moneymaverick
1,760655232011468800,jenpitt009
2,568457152,agotti22
3,1014584849544499201,ParetoParrot
4,981076429453471744,ohmybit_EN


# Get the friends of a user 

In [18]:
friends = api.friends_ids(screen_name = 'mikekobe0')
print ("This user follows", len(friends), "users.")

This user follows 2407 users.


In [19]:
df_following = pd.DataFrame(columns = ['user_number'], data = friends)

In [20]:
df_following['user_name'] = df_following['user_number'].apply(get_user_name)

In [21]:
df_following.head()

,user_number,user_name
0,94840831,dannyboycane13
1,4170079229,swisscapitalag
2,492742094,Max_Scherzer
3,19670226,ADL
4,183767243,AJCGlobal


 # SQL Database

In [2]:
import psycopg2
import string
import time

In [3]:
try:
    connection = psycopg2.connect(user = "dustin",
                                  password = "breitner",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "tweets")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print (connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
# finally:
#     #closing database connection.
#         if(connection):
#             cursor.close()
#             connection.close()
#             print("PostgreSQL connection is closed")

{'user': 'dustin', 'dbname': 'tweets', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.3 (Ubuntu 12.3-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',) 



 # All Tweets Table

In [4]:
# sql_create_table = """
# CREATE TABLE all_tweets ( 
# tweets_texts VARCHAR, 
# timestamps DATE, 
# users VARCHAR);"""

# cursor.execute(sql_create_table)

In [5]:
# cursor.execute("""DROP TABLE all_tweets;""")

In [6]:
# connection.rollback()

In [20]:
tweet_texts = []
timestamps = []
users = []
usernames = [] 
try:
    for username in usernames:
        for tweet in tw.Cursor(api.user_timeline, screen_name=username, exclude_replies= True, count=10).items():  
            tweet_text = tweet.text.strip().translate(str.maketrans('', '', string.punctuation))
            timestamp = tweet.created_at  
            tweeter = tweet.user.screen_name 

            cursor.execute('''
            INSERT INTO all_tweets (tweets_texts, timestamps, users)
            VALUES('{}','{}','{}')'''.format(tweet_text, timestamp, tweeter))
            connection.commit()
except tw.TweepError:  
    time.sleep(60)

In [56]:
df_all_tweets = pd.read_sql_query('SELECT * FROM all_tweets;',con=connection)

In [57]:
df_all_tweets.to_csv('df_all_tweets.csv')

In [22]:
df_all_tweets

,tweets_texts,timestamps,users
0,Grateful every day for the Blessings of a Big ...,2020-06-22,J_No24
1,heck I just found out beethoven was black yest...,2020-06-19,J_No24
2,Look I’m the 1st to raise my hand for Police r...,2020-06-19,J_No24
3,RT ryansmithtv the uncomfortability they had w...,2020-06-11,J_No24
4,RT BuffaloBills Josh Norman is using his platf...,2020-06-11,J_No24
...,...,...,...
322798,This is why I dont kick 😂😂 httpstcoEbF6THqFfZ,2016-06-27,D_Perryman52
322799,Whats going on httpstcovMyddtj2Id,2016-06-25,D_Perryman52
322800,Im good thanks for asking httpstcoDRuXzDXQet,2016-06-25,D_Perryman52
322801,Got me in my backyard fist pumping ✊🏽 httpstco...,2016-06-24,D_Perryman52


In [23]:
all_user_tweets = []
all_users = []

for user in df_all_tweets['users'].unique():
    user_df = df_all_tweets[df_all_tweets['users']==user]
    user_tweets_concat = ' '.join(user_df['tweets_texts'])
    all_user_tweets.append(user_tweets_concat)
    all_users.append(user)

In [39]:
df_clean = pd.DataFrame({'user': all_users, 
                         'concat_tweets': all_user_tweets})

In [40]:
df_clean.head()

,user,concat_tweets
0,J_No24,Grateful every day for the Blessings of a Big ...
1,SimplyAJ10,RT SimplyAJ10 Epicness httpstconqfUtKOsCF Epic...
2,TomBrady,httpstcoP3mmWBb76V We miss you already Kobe ❤️...
3,ochocinco,🦍 httpstcoQchtnLH6Ow 💎 httpstcoTHpQenvZxt 💎 ht...
4,RobGronkowski,Merry Christmas Thank you to our official food...


### TO DO

* clean text (move to lowercase, remove punctuation, remove stopwords, remove links - http) - DONE
* convert each user to one row (all texts combined) - DONE
* wordcloud to analyze common words for entire corpus (see if you need custom stop words). if this takes a long time don't worry about it now.
* use tf-idf to vectorize - DONE

later:
* MVP recommnedation engine model (nearest neighbors)
* more intense text cleaning (misspellings, custom stop words, etc)
* explore other vectorization methods (word2vec etc)

In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

In [42]:
#lowercase
df_clean['concat_tweets']=df_clean['concat_tweets'].str.lower()

In [43]:
#remove punctuation

def remove_punctuations(text):
    punct =[]
    punct += list(string.punctuation)
    punct += '’'
    punct.remove("'")
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

df_clean['concat_tweets'] = df_clean['concat_tweets'].apply(remove_punctuations)

In [44]:
#remove hyperlinks
df_clean['concat_tweets'] = df_clean['concat_tweets'].str.replace('http\S+|www.\S+', '', case=False)

In [45]:
df_clean['concat_tweets'] = df_clean['concat_tweets'].str.replace('\d+', '', regex=True)

In [31]:
df_clean.head()

,user,concat_tweets
0,J_No24,grateful every day for the blessings of a big ...
1,SimplyAJ10,rt simplyaj epicness epicness rt itsgabriell...
2,TomBrady,we miss you already kobe ❤️❤️🙏🏼🙏🏼 happy birth...
3,ochocinco,🦍 💎 💎 yahoo boy it s in the dna 🧬 how doe...
4,RobGronkowski,merry christmas thank you to our official food...


In [34]:
#TF-IDF

In [47]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [48]:
#lemmatize text
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

vectorizer = TfidfVectorizer(stop_words='english', min_df=0.02, tokenizer=tokenize_and_lemmatize)
vectors = vectorizer.fit_transform(df_clean['concat_tweets'])

In [49]:
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [50]:
df_featurized = pd.concat([df_clean, df_tfidf], axis=1).drop(['concat_tweets'], axis=1)

In [51]:
df_featurized.head()

,user,aa,aaa,aac,aaf,aafexpress,aaliyah,aampm,aampt,aap,...,🧘🏾‍♂️,🧟‍♂️,🧠,🧡,🧡🧡🧡,🧢,🧥,🧨,🧹,🩸
0,J_No24,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,SimplyAJ10,0.0,0.003172,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.002441,0.0,0.000000,0.000000,0.0,0.003547,0.000000
2,TomBrady,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,ochocinco,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.003437,...,0.0,0.0,0.0,0.015356,0.0,0.046167,0.009977,0.0,0.000000,0.013749
4,RobGronkowski,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


### TO DO

Additional text cleaning (don't spend too much time)
* stem/lemmatize - DONE
* remove numbers - DONE
* look at charts or wordclouds of top words, see if anything else should be cleaned up

* start building rec engine (get cosine sims, find most similar users, maybe use KNN) - google content based filtering with text

Later
* 

In [34]:
df_all_tweets.users.value_counts()

KB31_Era           8611
BenjaminSWatson    5863
DeionSanders       5818
LeVeonBell         5243
MarcedesLewis89    4248
                   ... 
stevenridley         42
MiamiDolphins         9
cjwentz               5
SportsGuy33           4
PatrickMahones        1
Name: users, Length: 156, dtype: int64

In [24]:
df_all_tweets.loc[df_all_tweets['users'] == 'M_Ryan02']

,tweets_texts,timestamps,users
314007,Clear my schedule httpstcoIblP8msYDp,2016-02-11,M_Ryan02
314103,Thoughts and prayers with AMB httpstco6LcBUNDy1W,2016-02-09,M_Ryan02
314154,VoteJUp,2014-07-10,M_Ryan02
314566,Good spot to spend the weekend httptcoVDVdfcFgeg,2014-06-15,M_Ryan02
315309,Having a great time at my ProCamps Thanks Publ...,2014-06-13,M_Ryan02
...,...,...,...
316314,Goal brian Boyle 30 rangers bchockey,2012-04-13,M_Ryan02
316315,Thanks to all of you guys welcoming me to twit...,2012-04-12,M_Ryan02
316316,“DustinCSmith This is by far my favorite pictu...,2012-04-12,M_Ryan02
316317,So I guess this is how twitter works Need some...,2012-04-12,M_Ryan02


 # All Followers Table

In [8]:
sql_create_table = """
CREATE TABLE all_followers ( 
users VARCHAR,  
followers VARCHAR);"""

# cursor.execute(sql_create_table)

In [5]:
# cursor.execute("""DROP TABLE all_followers;""")

In [6]:
# connection.rollback()

In [14]:
list_of_users = []
list_of_followers = []


usernames = []
try:   
    for username in usernames:    
        follower_numbers = api.followers_ids(screen_name=username, retry_errors=set([50]))

        cursor.execute('''
        INSERT INTO all_followers (users, followers)
        VALUES('{}','{}')'''.format(username, follower_numbers))
        connection.commit()
except tw.TweepError:  
    time.sleep(60)

In [53]:
df_all_followers = pd.read_sql_query('SELECT * FROM all_followers;',con=connection)

In [55]:
df_all_followers.to_csv('df_all_folllowers.csv')

In [8]:
df_all_followers

,users,followers
0,J_No24,"[1280557285434146816, 1280503002634354691, 128..."
1,SimplyAJ10,"[1191024377413931008, 1263958561677365248, 128..."
2,TomBrady,"[1263892928906305538, 1280574432268992512, 173..."
3,ochocinco,"[903980156179677184, 587066941, 1007764123, 12..."
4,RobGronkowski,"[1280574432268992512, 1231304902523129856, 927..."
...,...,...
151,GB_CoachMont,"[1280003121155059712, 1280179841678147586, 124..."
152,M_Ryan02,"[1279792674745266183, 1280602367239618560, 291..."
153,LarryFitzgerald,"[1280573827550978049, 794720842529832960, 1280..."
154,Linsley71,"[1280588280384323584, 30109599, 3029256646, 25..."


In [9]:
new_df_all_followers = df_all_followers.assign(followers=df_all_followers.followers.str.split(",")).explode('followers').reset_index(drop=True)

In [10]:
new_df_all_followers.followers = new_df_all_followers.followers.str.replace('[', '').str.replace(']', '')

In [11]:
new_df_all_followers

,users,followers
0,J_No24,1280557285434146816
1,J_No24,1280503002634354691
2,J_No24,1280434657637462016
3,J_No24,1280480053755854849
4,J_No24,163961043
...,...,...
746157,D_Perryman52,1302161292
746158,D_Perryman52,3247466640
746159,D_Perryman52,939329014740680704
746160,D_Perryman52,858332273934041088


# Users and Followers Table

In [18]:
# sql_create_table = """
# CREATE TABLE users_and_followers( 
# username VARCHAR, 
# follower_id VARCHAR, 
# follower_name VARCHAR);"""

# cursor.execute(sql_create_table)

In [17]:
# cursor.execute("""DROP TABLE users_and_followers;""")

In [16]:
connection.rollback()

In [25]:
new_df_all_followers.head()

,users,followers
0,J_No24,1280557285434146816
1,J_No24,1280503002634354691
2,J_No24,1280434657637462016
3,J_No24,1280480053755854849
4,J_No24,163961043


In [26]:
for index, row in new_df_all_followers.iterrows():
    username = row['users']
    follower_id = row['followers']
    print(username, follower_id)

J_No24 1280557285434146816
J_No24  1280503002634354691
J_No24  1280434657637462016
J_No24  1280480053755854849
J_No24  163961043
J_No24  876293763810947072
J_No24  1277286734183772161
J_No24  1280182170540179456
J_No24  1131729540504608768
J_No24  2563444516
J_No24  1729346402
J_No24  1280151541391986689
J_No24  1031837424178089984
J_No24  1058000858640728064
J_No24  1112344919555887105
J_No24  1097368808296235008
J_No24  2716587749
J_No24  1258974009116549121
J_No24  835933015192764417
J_No24  1015392791403384833
J_No24  1277863793763258375
J_No24  1279454328588247041
J_No24  1279409182303805440
J_No24  1171954461994098688
J_No24  34307129
J_No24  1023167466
J_No24  523343891
J_No24  3087178828
J_No24  3228163839
J_No24  1189952902628032514
J_No24  1234675512720678912
J_No24  1275244124556472321
J_No24  27006306
J_No24  362202959
J_No24  999121697272836099
J_No24  836806873626071040
J_No24  1276074905579204609
J_No24  1888273315
J_No24  19138824
J_No24  1279067454107529227
J_No24  215

J_No24  1267699548010487808
J_No24  940771436452511745
J_No24  285341109
J_No24  797999718727110657
J_No24  1129298017444683776
J_No24  1239965679022788609
J_No24  478958882
J_No24  1212021431342551040
J_No24  1347683610
J_No24  1017574626648645633
J_No24  1230497571812052992
J_No24  438407182
J_No24  27425842
J_No24  1267825698539843588
J_No24  815610572
J_No24  1267286457997238273
J_No24  826609551852052480
J_No24  1234294442380709888
J_No24  1243303425158365184
J_No24  1267623411091734529
J_No24  1033311737502728192
J_No24  1267120212383600640
J_No24  55190188
J_No24  1267920083420143626
J_No24  1071232482664742912
J_No24  1266815057423409155
J_No24  1166310260136693761
J_No24  1262214625082343425
J_No24  302438318
J_No24  2506172628
J_No24  161503085
J_No24  17792615
J_No24  1249123488314556421
J_No24  219779530
J_No24  1267873590977986568
J_No24  232051867
J_No24  106630911
J_No24  1259841091299422209
J_No24  4212795134
J_No24  944664017129189376
J_No24  1367886630
J_No24  1174507

J_No24  490704991
J_No24  73239106
J_No24  754484642598772736
J_No24  242417656
J_No24  386731514
J_No24  506175676
J_No24  1244713661626830854
J_No24  1242620344508329984
J_No24  111908273
J_No24  176603028
J_No24  3103298403
J_No24  1354543040
J_No24  1178776724286377985
J_No24  3412048300
J_No24  338624488
J_No24  537948396
J_No24  464316677
J_No24  220918266
J_No24  772828014
J_No24  229161933
J_No24  74753861
J_No24  262958923
J_No24  1178076799592538112
J_No24  34056383
J_No24  2304721078
J_No24  347325081
J_No24  635617478
J_No24  920831022840901632
J_No24  1849299038
J_No24  25339088
J_No24  2922420538
J_No24  225871611
J_No24  2416340411
J_No24  983366267196313601
J_No24  119046037
J_No24  627830476
J_No24  710237121882284033
J_No24  887932207
J_No24  472574289
J_No24  264924453
J_No24  326802850
J_No24  1134192971052716033
J_No24  924273800556044289
J_No24  2345169343
J_No24  989645466123096064
J_No24  875968874
J_No24  866877936636370945
J_No24  45171466
J_No24  3597860776
J

J_No24  2773593484
J_No24  84207708
J_No24  16214427
J_No24  51492081
J_No24  293222197
J_No24  826784657014452224
J_No24  160189601
J_No24  898587575992102943
J_No24  152330034
J_No24  60460405
J_No24  325304722
J_No24  406638284
J_No24  35279841
J_No24  4163594637
J_No24  3907982240
J_No24  1003007615138492417
J_No24  1261795903
J_No24  910502163562090496
J_No24  1227737214576185346
J_No24  18279803
J_No24  167913085
J_No24  287964115
J_No24  4367481436
J_No24  4493466207
J_No24  43117138
J_No24  253117056
J_No24  1053805819
J_No24  79532004
J_No24  265692463
J_No24  744034285
J_No24  239887925
J_No24  87236086
J_No24  385477400
J_No24  21836978
J_No24  1247562669202108418
J_No24  1247758383790186502
J_No24  85174939
J_No24  1963180543
J_No24  88758493
J_No24  1232139759444807686
J_No24  1233415588057358338
J_No24  1227950981377404934
J_No24  917798367614214144
J_No24  1214958375001309185
J_No24  1247466746212122641
J_No24  1147408189362753536
J_No24  36453567
J_No24  124662531886301

J_No24  971452111635517440
J_No24  1215083735290449920
J_No24  4916425222
J_No24  1223783478791221250
J_No24  624889548
J_No24  780575784700305410
J_No24  282199045
J_No24  1475598528
J_No24  1225448954504130560
J_No24  3088886014
J_No24  2676666243
J_No24  3390451451
J_No24  1234394200558440448
J_No24  43955652
J_No24  773234491183824896
J_No24  376292210
J_No24  972586633051721729
J_No24  3228332856
J_No24  74617226
J_No24  328682885
J_No24  257587813
J_No24  2345406551
J_No24  721500038179000324
J_No24  26910286
J_No24  31190324
J_No24  2345571502
J_No24  3185488421
J_No24  816073244704931840
J_No24  390948309
J_No24  1157927131
J_No24  706240601834979328
J_No24  69012749
J_No24  1179427680103653376
J_No24  945709625940332545
J_No24  166348853
J_No24  786937910524252160
J_No24  335541462
J_No24  277137106
J_No24  493448553
J_No24  445023233
J_No24  1080160140521660416
J_No24  625118566
J_No24  256158964
J_No24  2380753816
J_No24  119248057
J_No24  54260075
J_No24  123971204803824845

...

CMC_22  1961924324
CMC_22  21700194
CMC_22  1258213700776816640
CMC_22  1176813844812967937
CMC_22  2379808368
CMC_22  1663653842
CMC_22  825140940847329284
CMC_22  187686368
CMC_22  1240050314503389185
CMC_22  41662157
CMC_22  1264215874678018049
CMC_22  4581074953
CMC_22  6057252
CMC_22  197598241
CMC_22  35760835
CMC_22  1523707892
CMC_22  1039179155957526528
CMC_22  1087163839
CMC_22  2710864934
CMC_22  248672689
CMC_22  701867177322926080
CMC_22  1176598882555482113
CMC_22  1280839538
CMC_22  1931285130
CMC_22  888380600050229249
CMC_22  2241314868
CMC_22  819348412432781312
CMC_22  1169006544433930245
CMC_22  2561975031
CMC_22  164118952
CMC_22  1183859749080752128
CMC_22  366105628
CMC_22  4463594867
CMC_22  2970060460
CMC_22  1080640414326108160
CMC_22  1107955766
CMC_22  1063873183743451136
CMC_22  467164222
CMC_22  1240795246453231616
CMC_22  1263843652079456257
CMC_22  141423498
CMC_22  2256456702
CMC_22  3323440531
CMC_22  1119343699337437184
CMC_22  1626104100
CMC_22  3330

KingHenry_2  1190198968216674305
KingHenry_2  816433581274722304
KingHenry_2  1279660044661911552
KingHenry_2  1279656002481082368
KingHenry_2  425328375
KingHenry_2  826964387717984257
KingHenry_2  1279652799219933184
KingHenry_2  363888572
KingHenry_2  1279230197062004736
KingHenry_2  266369926
KingHenry_2  1465995343
KingHenry_2  1673925409
KingHenry_2  3036860658
KingHenry_2  1274082780297809920
KingHenry_2  1118679577335934976
KingHenry_2  1271346820023713793
KingHenry_2  2473093416
KingHenry_2  1279605763678363649
KingHenry_2  1051183080092954624
KingHenry_2  1279610062433067014
KingHenry_2  1279479008678289409
KingHenry_2  241712934
KingHenry_2  894722996560068612
KingHenry_2  1279601667776937985
KingHenry_2  1015376966189506561
KingHenry_2  466185192
KingHenry_2  1171667557297852417
KingHenry_2  1267522612697145345
KingHenry_2  1279586685999693824
KingHenry_2  526798600
KingHenry_2  1279579968524169216
KingHenry_2  249357893
KingHenry_2  1278782884866908162
KingHenry_2  1148706

KingHenry_2  1275418729741004800
KingHenry_2  1278138935605108738
KingHenry_2  1270052399009447936
KingHenry_2  1239957184156499968
KingHenry_2  1278124116764241920
KingHenry_2  1252305361425571844
KingHenry_2  1278113582559956995
KingHenry_2  1273972679285055489
KingHenry_2  1274022107953889287
KingHenry_2  61142075
KingHenry_2  541862612
KingHenry_2  1193343610365915136
KingHenry_2  2740457579
KingHenry_2  1278070734028300288
KingHenry_2  1278053590565781505
KingHenry_2  1278047020603842563
KingHenry_2  1271561721681973250
KingHenry_2  1278045630405898245
KingHenry_2  1278036059243327491
KingHenry_2  808817413953294336
KingHenry_2  1278018656866390016
KingHenry_2  1278020019033722881
KingHenry_2  1278014322908815360
KingHenry_2  2256737174
KingHenry_2  1277991500157276160
KingHenry_2  1277976599745314816
KingHenry_2  127733071
KingHenry_2  1277969015218921478
KingHenry_2  1269750528852070403
KingHenry_2  1276884897483960321
KingHenry_2  881435306
KingHenry_2  1277643186832388097
King

KingHenry_2  233239638
KingHenry_2  2720101257
KingHenry_2  2320274510
KingHenry_2  561595415
KingHenry_2  290148322
KingHenry_2  1017187601864945665
KingHenry_2  150102492
KingHenry_2  2634199962
KingHenry_2  822425438
KingHenry_2  2997361993
KingHenry_2  2926334986
KingHenry_2  1222203065866104832
KingHenry_2  1089180443587592192
KingHenry_2  737829092578463744
KingHenry_2  931649401109516288
KingHenry_2  1107363875115155457
KingHenry_2  56259177
KingHenry_2  1199332638
KingHenry_2  1272272391499022343
KingHenry_2  2471517752
KingHenry_2  531980220
KingHenry_2  51013127
KingHenry_2  948580383507001344
KingHenry_2  167163796
KingHenry_2  810843528
KingHenry_2  701037540262936576
KingHenry_2  334771741
KingHenry_2  520519648
KingHenry_2  1190283656406765568
KingHenry_2  80381731
KingHenry_2  4018470792
KingHenry_2  1276979787366633474
KingHenry_2  998893110
KingHenry_2  1167513317327478784
KingHenry_2  799075434801623040
KingHenry_2  189053893
KingHenry_2  717265748
KingHenry_2  707349

KeyboardInterrupt: 

In [21]:
try:
    for index, row in new_df_all_followers.iterrows():
        username = row['users']
        follower_id = row['followers']
        
        # if no row with username and follower id exists in SQL table...
        
        follower_name = api.get_user(follower_id).screen_name
    
        cursor.execute('''
        INSERT INTO users_and_followers (username, follower_id, follower_name)
        VALUES('{}','{}','{}')'''.format(username, follower_id, follower_name))
        connection.commit()
        
        # else:
            #continue
except tw.TweepError:
    time.sleep(60)

In [22]:
df_users_and_followers = pd.read_sql_query('SELECT * FROM users_and_followers;',con=connection)

In [23]:
df_users_and_followers

,username,follower_id,follower_name
0,J_No24,1280557285434146816,yipyapiyap
1,J_No24,1280503002634354691,NoureenRahman5
2,J_No24,1280434657637462016,Angelic74974691
3,J_No24,1280480053755854849,weirdo32189661
4,J_No24,163961043,abavery6
...,...,...,...
539,J_No24,1679257009,paulconmcguire
540,J_No24,1269009051175858177,katelyn_roke
541,J_No24,3014671065,PittsyNFL
542,J_No24,1151625874367754240,MrOn2u
